In [ ]:
!pip install torchtext==0.6.0

!pip install -U pip setuptools wheel

!pip install -U spacy==2.3.5

!python -m spacy download en_core_web_sm

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data import Field, TabularDataset, BucketIterator, LabelField
import spacy
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import nltk
from nltk.tokenize import word_tokenize
import time

In [ ]:
!pip install gdown==4.6.0
!gdown --folder https://drive.google.com/drive/u/1/folders/15Wn46r7gidaiZbx2ArFYsd7rjYH4y7JM

In [ ]:
# Set random seed for reproducibility
seed = 1234
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# load the liar dataset
csv_path_liar_train = os.path.join( '/content', 'liar_dataset', 'train.tsv')
csv_path_liar_test = os.path.join( '/content', 'liar_dataset', 'test.tsv')

df_liar_train = pd.read_csv(csv_path_liar_train, sep='\t', header=None)
df_liar_test = pd.read_csv(csv_path_liar_test, sep='\t', header=None)

df_liar_train.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']
df_liar_test.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']

df_liar_train = df_liar_train[['label', 'statement']]
df_liar_test = df_liar_test[['label', 'statement']]
df_liar_train = df_liar_train.dropna()
df_liar_test = df_liar_test.dropna()


# save the train and test sets to csv files
df_liar_train.to_csv('train.csv', index=False)
df_liar_test.to_csv('test.csv', index=False)

In [ ]:
nlp = spacy.load("en_core_web_sm")
def tokenize_spacy(text):
    return [token.text for token in nlp(text)]


# define the fields
TEXT = Field(tokenize=tokenize_spacy, include_lengths=True, batch_first=True)
LABEL = LabelField(dtype=torch.float, batch_first = True)

fields = [('text', TEXT), ('label', LABEL)]
train_data, test_data = TabularDataset.splits(
    path='./',
    train='train.csv',
    test='test.csv',
    format='csv',
    skip_header=True,
    fields=[('label', LABEL), ('text', TEXT)]
)

# split the train data into train and validation sets

train_data, valid_data = train_data.split(random_state=random.seed(seed))

# build the vocabulary
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                    max_size=MAX_VOCAB_SIZE,
                    vectors="glove.6B.100d",
                    unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)


# Define iterator
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    device=device
    )

.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 399999/400000 [00:21<00:00, 18863.02it/s]


In [ ]:
# Define the transformer model
class Transformer(nn.Module):
    def __init__(self, input_dim, emb_dim, nhead, hid_dim, n_layers, output_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.transformer = nn.Transformer(
            d_model=emb_dim,
            nhead=nhead,
            num_encoder_layers=n_layers,
            num_decoder_layers=n_layers,
            dim_feedforward=hid_dim,
            dropout=dropout
        )
        self.fc = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
      embedded = self.embedding(text)

      # If the input is sparse, convert it to a dense tensor
      if isinstance(embedded, torch.sparse.FloatTensor):
          embedded = embedded.to_dense()

      embedded = embedded.permute(1, 0, 2)

      output = self.transformer(embedded, embedded)
      output = output.mean(dim=0)
      output = self.fc(output)

      return F.log_softmax(output, dim=1)

In [ ]:
# Initialize the model, optimizer, and loss function
INPUT_DIM = len(TEXT.vocab)
EMB_DIM = 100
NHEAD = 4
HID_DIM = 256
N_LAYERS = 2
OUTPUT_DIM = 6
DROPOUT = 0.5

model = Transformer(INPUT_DIM, EMB_DIM, NHEAD, HID_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# Send the model to GPU if available

model = model.to(device)
criterion = criterion.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Training loop
# define the training function

def categorical_accuracy(preds, y):
    top_pred = preds.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    return correct.float() / y.shape[0]

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in tqdm(iterator):

        optimizer.zero_grad()


        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label.long())

        acc = categorical_accuracy(predictions, batch.label.long())

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)
# Evaluation loop
def evaluate(model, iterator, criterion):

        epoch_loss = 0
        epoch_acc = 0

        all_predictions = []
        all_labels = []


        model.eval()

        with torch.no_grad():

            for batch in tqdm(iterator):

                text, text_lengths = batch.text

                predictions = model(text, text_lengths).squeeze(1)

                loss = criterion(predictions, batch.label.long())

                acc = categorical_accuracy(predictions, batch.label.long())

                epoch_loss += loss.item()
                epoch_acc += acc.item()

                # calculate precision, recall and f1 score
                y_pred = predictions.argmax(1, keepdim=True)
                y_pred = y_pred.squeeze(1)
                y_true = batch.label.long()
                #y_true = y_true.squeeze(1)

                all_predictions.extend(y_pred.cpu().numpy())
                all_labels.extend(batch.label.long().cpu().numpy())

        precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=True)
        recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=True)
        f1 = f1_score(all_labels, all_predictions, average='weighted')


        return epoch_loss / len(iterator), epoch_acc / len(iterator), precision, recall, f1

In [ ]:
# Training and evaluation
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train(model, train_iterator, optimizer, criterion)
    test_loss, test_accuracy, _, _, _ = evaluate(model, test_iterator, criterion)
    print(f'Epoch: {epoch+1}, Test Loss: {test_loss:.3f}, Test Accuracy: {test_accuracy:.3f}')

# Save the trained model
torch.save(model.state_dict(), 'fake_news_transformer_model.pth')

100%|██████████| 20/20 [00:00<00:00, 138.61it/s]


Epoch: 1, Test Loss: 1.761, Test Accuracy: 0.219


100%|██████████| 20/20 [00:00<00:00, 131.02it/s]


Epoch: 2, Test Loss: 1.804, Test Accuracy: 0.204


100%|██████████| 20/20 [00:00<00:00, 137.10it/s]


Epoch: 3, Test Loss: 1.802, Test Accuracy: 0.212


100%|██████████| 20/20 [00:00<00:00, 136.91it/s]


Epoch: 4, Test Loss: 1.827, Test Accuracy: 0.228


100%|██████████| 20/20 [00:00<00:00, 105.16it/s]


Epoch: 5, Test Loss: 1.987, Test Accuracy: 0.211


In [ ]:
# evaluate the model on the test set
test_loss, test_acc, precision, recall, f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Precision: {precision} | Recall: {recall} | F1: {f1}')

# save the results to a csv file
results = pd.DataFrame([[test_acc, precision, recall, f1]], columns=['accuracy', 'precision', 'recall', 'f1'], index=['Transformer'])
results.to_csv('results_liar_transformer.csv')

100%|██████████| 20/20 [00:00<00:00, 95.99it/s]

Test Loss: 1.987 | Test Acc: 21.08% | Precision: 0.16976947159985484 | Recall: 0.2107340173638516 | F1: 0.17456077812545948


In [ ]:
!pip install gdown==4.6.0
!gdown --folder https://drive.google.com/drive/u/1/folders/1wf7mFLCqQo0t802IDkZKMOinciUwohuR

In [ ]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

nltk.download('punkt')
def tokenize(text):
    return word_tokenize(text)



# define the fields
TEXT = Field(tokenize=tokenize, include_lengths=True, unk_token='<unk>', batch_first=True)
LABEL = LabelField(dtype=torch.float)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# load the welfake dataset

csv_path_welfake = os.path.join( '/content', 'WELFake', 'WELFake_Dataset.csv')

df = pd.read_csv(csv_path_welfake)
df = df.drop(['Unnamed: 0', 'title'], axis=1)
df.columns = ['text', 'label']
df['label'] = df['label'].replace('fake', 0)
df['label'] = df['label'].replace('real', 1)
df.to_csv('.//welfake.csv', index=False)
# drop the rows with np.nan values on text column
df = df.dropna(subset=['text'])
df = df[df['text'].str.len() > 30]

# split the dataset into train, validation and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED)


####### DEBUG

#train_df, test_df = train_test_split(test_df, test_size=0.2, random_state=SEED)

# save the train, validation and test sets as csv files
train_df.to_csv('.//welfake_train.csv', index=False)
test_df.to_csv('.//welfake_test.csv', index=False)

In [ ]:
# load the data
train_data, test_data = TabularDataset.splits(
    path='./',
    train='welfake_train.csv',
    test='welfake_test.csv',
    format='csv',
    fields=[('text', TEXT), ('label', LABEL)]
)

# split the train data into train and validation sets

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [ ]:
# build the vocabulary
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                    max_size=MAX_VOCAB_SIZE,
                    vectors="glove.6B.100d",
                    unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)



.vector_cache/glove.6B.zip: 862MB [02:42, 5.32MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19502.90it/s]


In [ ]:
# create the iterators
BATCH_SIZE = 16

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    device=device
    )

In [6]:
# Define the transformer model
class Transformer(nn.Module):
    def __init__(self, input_dim, emb_dim, nhead, hid_dim, n_layers, output_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.transformer = nn.Transformer(
            d_model=emb_dim,
            nhead=nhead,
            num_encoder_layers=n_layers,
            num_decoder_layers=n_layers,
            dim_feedforward=hid_dim,
            dropout=dropout
        )
        self.fc = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
      embedded = self.embedding(text)

      # If the input is sparse, convert it to a dense tensor
      if isinstance(embedded, torch.sparse.FloatTensor):
          embedded = embedded.to_dense()

      embedded = embedded.permute(1, 0, 2)

      output = self.transformer(embedded, embedded)
      output = output.mean(dim=0)
      output = self.fc(output).squeeze(dim=0)

      return output

In [ ]:
# Initialize the model, optimizer, and loss function
INPUT_DIM = len(TEXT.vocab)
EMB_DIM = 100
NHEAD = 4
HID_DIM = 256
N_LAYERS = 2
OUTPUT_DIM = 1
DROPOUT = 0.5

model = Transformer(INPUT_DIM, EMB_DIM, NHEAD, HID_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)
optimizer = optim.Adam(model.parameters(), lr = .001)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
criterion = nn.BCEWithLogitsLoss()

# Send the model to GPU if available

model = model.to(device)
criterion = criterion.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
# Training loop
# define the training function
def binary_accuracy(preds, y):
    threshold = 0.5
    binary_predictions = (preds > threshold).float()
    correct = (binary_predictions == y).float()
    #correct = (preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):


    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in tqdm(iterator):

        optimizer.zero_grad()


        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label.float())

        acc = binary_accuracy(predictions, batch.label.long())

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()


    return epoch_loss / len(iterator), epoch_acc / len(iterator)
# Evaluation loop
def evaluate(model, iterator, criterion):

        epoch_loss = 0
        epoch_acc = 0

        all_predictions = []
        all_labels = []


        model.eval()

        with torch.no_grad():

            for batch in tqdm(iterator):

                text, text_lengths = batch.text

                try:
                  predictions = model(text, text_lengths).squeeze(1)
                except:
                  predictions = model(text, text_lengths)
                loss = criterion(predictions, batch.label)

                acc = binary_accuracy(predictions, batch.label)

                epoch_loss += loss.item()
                epoch_acc += acc.item()

                # Convert probability scores to binary predictions using a threshold (e.g., 0.5)
                threshold = 0.5
                binary_predictions = (predictions > threshold).float()

                # calculate precision, recall and f1 score

                all_predictions.extend(binary_predictions.cpu().numpy())
                all_labels.extend(batch.label.long().cpu().numpy())


        precision = precision_score(all_labels, all_predictions,  zero_division=True)
        recall = recall_score(all_labels, all_predictions,  zero_division=True)
        f1 = f1_score(all_labels, all_predictions)


        return epoch_loss / len(iterator), epoch_acc / len(iterator), precision, recall, f1

def epoch_time(start_time, end_time):

        elapsed_time = end_time - start_time

        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))

        return elapsed_mins, elapsed_secs

In [ ]:
# train the model

N_EPOCHS = 4

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

            start_time = time.time()

            train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
            valid_loss, valid_acc, _, _, _ = evaluate(model, valid_iterator, criterion)

            end_time = time.time()

            epoch_mins, epoch_secs = epoch_time(start_time, end_time)

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), 'welfake-model.pt')

            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# evaluate the model on the test set

model.load_state_dict(torch.load('welfake-model.pt'))

100%|██████████| 1068/1068 [00:50<00:00, 21.18it/s]


Epoch: 01 | Epoch Time: 14m 25s
	Train Loss: 0.198 | Train Acc: 91.81%
	 Val. Loss: 0.178 |  Val. Acc: 92.97%


100%|██████████| 1068/1068 [00:50<00:00, 21.30it/s]


Epoch: 02 | Epoch Time: 15m 18s
	Train Loss: 0.099 | Train Acc: 96.43%
	 Val. Loss: 0.093 |  Val. Acc: 97.24%


100%|██████████| 1068/1068 [00:49<00:00, 21.61it/s]


Epoch: 03 | Epoch Time: 15m 29s
	Train Loss: 0.105 | Train Acc: 96.24%
	 Val. Loss: 0.302 |  Val. Acc: 93.22%


100%|██████████| 1068/1068 [00:49<00:00, 21.37it/s]

Epoch: 04 | Epoch Time: 14m 49s
	Train Loss: 0.258 | Train Acc: 88.04%
	 Val. Loss: 0.733 |  Val. Acc: 50.40%


<All keys matched successfully>

In [ ]:
# evaluate the model on the test set
test_loss, test_acc, precision, recall, f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Precision: {precision} | Recall: {recall} | F1: {f1}')

# save the results to a csv file
results = pd.DataFrame([[test_acc, precision, recall, f1]], columns=['accuracy', 'precision', 'recall', 'f1'], index=['Transformer'])
results.to_csv('results_WELFake_transformer.csv')

100%|██████████| 891/891 [00:47<00:00, 18.84it/s]

Test Loss: 0.093 | Test Acc: 97.15% | Precision: 0.97149313575598 | Recall: 0.9714907660978864 | F1: 0.9714564080718354


In [3]:
# Set random seed for reproducibility
seed = 1234
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# load the liar dataset
csv_path_liar_train = os.path.join( '/content', 'liar_dataset', 'train.tsv')
csv_path_liar_test = os.path.join( '/content', 'liar_dataset', 'test.tsv')

df_liar_train = pd.read_csv(csv_path_liar_train, sep='\t', header=None)
df_liar_test = pd.read_csv(csv_path_liar_test, sep='\t', header=None)

df_liar_train.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']
df_liar_test.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']

df_liar_train = df_liar_train[['label', 'statement']]
df_liar_test = df_liar_test[['label', 'statement']]
df_liar_train = df_liar_train.dropna()
df_liar_test = df_liar_test.dropna()

df_liar_train['label'] = df_liar_train['label'].replace(['mostly-true'], 'true')
df_liar_test['label'] = df_liar_test['label'].replace(['mostly-true'], 'true')
df_liar_train['label'] = df_liar_train['label'].replace(['half-true'], 'true')
df_liar_test['label'] = df_liar_test['label'].replace(['half-true'], 'true')
df_liar_train['label'] = df_liar_train['label'].replace(['barely-true'], 'false')
df_liar_test['label'] = df_liar_test['label'].replace(['barely-true'], 'false')
df_liar_train['label'] = df_liar_train['label'].replace(['pants-fire'], 'false')
df_liar_test['label'] = df_liar_test['label'].replace(['pants-fire'], 'false')


# save the train and test sets to csv files
df_liar_train.to_csv('train.csv', index=False)
df_liar_test.to_csv('test.csv', index=False)

In [5]:
nlp = spacy.load("en_core_web_sm")
def tokenize_spacy(text):
    return [token.text for token in nlp(text)]


# define the fields
TEXT = Field(tokenize=tokenize_spacy, include_lengths=True, batch_first=True)
LABEL = LabelField(dtype=torch.float, batch_first = True)

fields = [('text', TEXT), ('label', LABEL)]
train_data, test_data = TabularDataset.splits(
    path='./',
    train='train.csv',
    test='test.csv',
    format='csv',
    skip_header=True,
    fields=[('label', LABEL), ('text', TEXT)]
)

# split the train data into train and validation sets

train_data, valid_data = train_data.split(random_state=random.seed(seed))

# build the vocabulary
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                    max_size=MAX_VOCAB_SIZE,
                    vectors="glove.6B.100d",
                    unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)


# Define iterator
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    device=device
    )

.vector_cache/glove.6B.zip: 862MB [02:44, 5.25MB/s]                           
100%|█████████▉| 399999/400000 [00:21<00:00, 18956.16it/s]


In [19]:
# Initialize the model, optimizer, and loss function
INPUT_DIM = len(TEXT.vocab)
EMB_DIM = 100
NHEAD = 5
HID_DIM = 256
N_LAYERS = 2
OUTPUT_DIM = 1
DROPOUT = 0.6

model = Transformer(INPUT_DIM, EMB_DIM, NHEAD, HID_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)
optimizer = optim.Adam(model.parameters(), lr = .0001)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
criterion = nn.BCEWithLogitsLoss()

# Send the model to GPU if available

model = model.to(device)
criterion = criterion.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [20]:
# train the model

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

            start_time = time.time()

            train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
            valid_loss, valid_acc, _, _, _ = evaluate(model, valid_iterator, criterion)

            end_time = time.time()

            epoch_mins, epoch_secs = epoch_time(start_time, end_time)

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), 'LIAR-BINARY-model.pt')

            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# evaluate the model on the test set

model.load_state_dict(torch.load('LIAR-BINARY-model.pt'))

100%|██████████| 48/48 [00:00<00:00, 215.96it/s]


Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.688 | Train Acc: 55.98%
	 Val. Loss: 0.679 |  Val. Acc: 56.58%


100%|██████████| 48/48 [00:00<00:00, 212.12it/s]


Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.686 | Train Acc: 56.03%
	 Val. Loss: 0.687 |  Val. Acc: 56.54%


100%|██████████| 48/48 [00:00<00:00, 197.18it/s]


...

100%|██████████| 48/48 [00:00<00:00, 216.46it/s]


Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 0.668 | Train Acc: 57.03%
	 Val. Loss: 0.870 |  Val. Acc: 58.33%


100%|██████████| 48/48 [00:00<00:00, 213.07it/s]

Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: 0.666 | Train Acc: 57.88%
	 Val. Loss: 0.865 |  Val. Acc: 58.30%


<All keys matched successfully>

In [21]:
# evaluate the model on the test set
test_loss, test_acc, precision, recall, f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Precision: {precision} | Recall: {recall} | F1: {f1}')

# save the results to a csv file
results = pd.DataFrame([[test_acc, precision, recall, f1]], columns=['accuracy', 'precision', 'recall', 'f1'], index=['Transformer'])
results.to_csv('results_LIAR_BINARY_transformer.csv')

100%|██████████| 20/20 [00:00<00:00, 84.48it/s]

Test Loss: 0.685 | Test Acc: 56.38% | Precision: 0.0 | Recall: 0.0 | F1: 0.0


In [22]:
!gdown 1S2ihcguQAX6mU7WB7XFhFiWzBjlbFd2h

Downloading...
From: https://drive.google.com/uc?id=1S2ihcguQAX6mU7WB7XFhFiWzBjlbFd2h
To: /content/welfake-model-transformer.pt
100% 11.8M/11.8M [00:00<00:00, 70.9MB/s]


In [23]:
# Set random seed for reproducibility
seed = 1234
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
# Define the transformer model
class Transformer(nn.Module):
    def __init__(self, input_dim, emb_dim, nhead, hid_dim, n_layers, output_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.transformer = nn.Transformer(
            d_model=emb_dim,
            nhead=nhead,
            num_encoder_layers=n_layers,
            num_decoder_layers=n_layers,
            dim_feedforward=hid_dim,
            dropout=dropout
        )
        self.fc = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
      embedded = self.embedding(text)

      # If the input is sparse, convert it to a dense tensor
      if isinstance(embedded, torch.sparse.FloatTensor):
          embedded = embedded.to_dense()

      embedded = embedded.permute(1, 0, 2)

      output = self.transformer(embedded, embedded)
      output = output.mean(dim=0)
      output = self.fc(output).squeeze(dim=0)

      return output

In [30]:
# Initialize the model, optimizer, and loss function
INPUT_DIM = 25002
EMB_DIM = 100
NHEAD = 4
HID_DIM = 256
N_LAYERS = 2
OUTPUT_DIM = 1
DROPOUT = 0.5

model = Transformer(INPUT_DIM, EMB_DIM, NHEAD, HID_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)
optimizer = optim.Adam(model.parameters(), lr = .001)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
criterion = nn.BCEWithLogitsLoss()

# Send the model to GPU if available

model = model.to(device)
criterion = criterion.to(device)

In [31]:
model.load_state_dict(torch.load('welfake-model-transformer.pt'))

<All keys matched successfully>

In [33]:
# evaluate the model on the test set
test_loss, test_acc, precision, recall, f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Precision: {precision} | Recall: {recall} | F1: {f1}')

# save the results to a csv file
results = pd.DataFrame([[test_acc, precision, recall, f1]], columns=['accuracy', 'precision', 'recall', 'f1'], index=['Transformer-Pretrained'])
results.to_csv('results_LIAR_BINARY_transformer_PRETRAINED.csv')

100%|██████████| 20/20 [00:00<00:00, 89.69it/s]

Test Loss: 3.375 | Test Acc: 50.19% | Precision: 0.37962962962962965 | Recall: 0.2224231464737794 | F1: 0.2805017103762828
